# Playlist analyzer

## Import the spotify api

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
))

playlists = spotify.user_playlists('')
name = input("Enter playlist name: ")


# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(playlist_id):
    results = spotify.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = get_playlist_tracks(playlist_id)

/Users/liamkrenz/Desktop/ATCS/Final Project/ATCSFinalProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Extract song data on each song

In [2]:
playlist = []
names = []

for i in playlist_data : 
    names.append(i["track"]["name"])
    playlist.append(spotify.audio_features(i["track"]["id"])[0])

print(playlist[1])


{'danceability': 0.805, 'energy': 0.707, 'key': 6, 'loudness': -3.758, 'mode': 0, 'speechiness': 0.0353, 'acousticness': 0.142, 'instrumentalness': 3.71e-05, 'liveness': 0.563, 'valence': 0.744, 'tempo': 122.014, 'type': 'audio_features', 'id': '4eMKD8MRroxCqugpsxCCNb', 'uri': 'spotify:track:4eMKD8MRroxCqugpsxCCNb', 'track_href': 'https://api.spotify.com/v1/tracks/4eMKD8MRroxCqugpsxCCNb', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4eMKD8MRroxCqugpsxCCNb', 'duration_ms': 228197, 'time_signature': 4}


### Convert the new data into a dataframe including the name of each song

In [3]:
collumns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"]
selected_columns = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

df = pd.DataFrame(playlist, columns=collumns)
df = df[selected_columns]
df["name"] = names

scaler = MinMaxScaler()
scaler = scaler.fit(df[["tempo"]])

df["tempo"] = pd.DataFrame(scaler.transform(df[["tempo"]]))
df





,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name
0,0.314,0.461,-8.274,0.0280,0.04010,0.000935,0.2010,0.266,0.831225,Knew You
1,0.805,0.707,-3.758,0.0353,0.14200,0.000037,0.5630,0.744,0.442152,MOJABI GHOST
2,0.734,0.846,-3.425,0.0489,0.16600,0.000002,0.2350,0.604,0.247490,Nostálgico
3,0.753,0.678,-5.421,0.0644,0.08500,0.000002,0.1300,0.583,0.231098,Blank Space
4,0.670,0.858,-6.528,0.0439,0.00354,0.000013,0.0687,0.539,0.409652,I Wish You Would (Taylor's Version)
...,...,...,...,...,...,...,...,...,...,...
154,0.701,0.944,-2.986,0.0455,0.07550,0.000000,0.6010,0.792,0.498787,Uptown Girl
155,0.856,0.609,-7.223,0.0824,0.00801,0.000082,0.0344,0.928,0.385137,Uptown Funk (feat. Bruno Mars)
156,0.514,0.730,-5.934,0.0598,0.00146,0.000095,0.0897,0.334,0.839713,Blinding Lights
157,0.492,0.810,-5.315,0.0847,0.04160,0.003790,0.1170,0.245,0.668885,Dog Days Are Over


## Find the distance of each song to the central point of the playlist's cluster

In [4]:
model = KMeans(n_clusters=1)
model.fit(df[selected_columns])


centroid = model.cluster_centers_[0]
distances = []

for i in range(len(df)) :
    distance = cdist([df[selected_columns].iloc[i]], [centroid], metric="euclidean")
    distances.append(distance[0][0])

df["distance"] = distances

Q1 = df["distance"].quantile(0.25)
Q3 = df['distance'].quantile(0.75)

max = Q3 + (Q3-Q1)*1.5
print(max)

df.loc[df["distance"] > max]



4.731741198633519


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name,distance
51,0.566,0.366,-12.808,0.0280,0.113,0.18100,0.155,0.237,0.506634,we fell in love in october,6.704960
80,0.809,0.463,-11.377,0.0396,0.256,0.08500,0.109,0.364,0.360727,Passionfruit,5.270838
123,0.453,0.350,-11.639,0.0425,0.827,0.00122,0.144,0.279,0.036428,Arcade,5.586864
158,0.226,0.280,-11.460,0.0316,0.949,0.08870,0.315,0.625,0.221011,The Star Spangled Banner (United States of Ame...,5.431824
